<a href="https://colab.research.google.com/github/nishanthjois/2021_DeepLearning/blob/main/8_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install torch-geometric

     |████████████████████████████████| 7.9 MB 2.1 MB/s 
     |████████████████████████████████| 3.5 MB 2.8 MB/s 
     |████████████████████████████████| 370 kB 5.1 MB/s 
     |████████████████████████████████| 482 kB 46.4 MB/s 
     |████████████████████████████████| 41 kB 581 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.3-py3-none-any.whl size=581968 sha256=47244049d64f655ebfb66e4b3f3d0f1314ab833618091aa7059498fc73781af4
  Stored in directory: /root/.cache/pip/wheels/c3/2a/58/87ce0508964d4def1aafb92750c4f3ac77038efd1b9a89dcf5
Successfully built torch-geometric


In [1]:
!mkdir data

In [2]:
#download data
!apt-get install p7zip
!curl -Lo yoochoose-data.7z https://s3-eu-west-1.amazonaws.com/yc-rdata/yoochoose-data.7z
!7z x yoochoose-data.7z

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip is already the newest version (16.02+dfsg-6).
p7zip set to manually installed.
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-

In [4]:
!mv yoo* data/

In [18]:
%%writefile dataloader.py
import torch
from torch_geometric.data import Dataset


# The class inherits the base class Dataset from pytorch
class LoadData(Dataset):  # for training/testing
    def __init__(self, data_path):
        super(LoadData, self).__init__()
        self.data = torch.load(data_path)

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

Writing dataloader.py


In [17]:
%%writefile gnn.py
# # cimport matplotlib.pyplot as plt
#import osmnx as ox
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.data import InMemoryDataset
from torch_geometric.nn import TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score


#from data_loader import YooChooseDataset

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from torch_geometric.data import InMemoryDataset
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, SAGEConv, SGConv, SplineConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F

np.random.seed(42)

datatset_size = 10000

clicks_df = pd.read_csv('./data/yoochoose-clicks.dat', header=None)
clicks_df.columns = ['session_id', 'timestamp', 'item_id', 'category']
print(clicks_df.head(5))

buy_df = pd.read_csv('./data/yoochoose-buys.dat', header=None)
buy_df.columns = ['session_id', 'timestamp', 'item_id', 'price', 'quantity']
print(buy_df.head(5))

item_encoder = LabelEncoder()
clicks_df['item_id'] = item_encoder.fit_transform(clicks_df.item_id)
print(clicks_df.head())

#randomly sample a couple of them
sampled_session_id = np.random.choice(clicks_df.session_id.unique(), datatset_size, replace=False)
clicks_df = clicks_df.loc[clicks_df.session_id.isin(sampled_session_id)]
print(clicks_df.nunique())

clicks_df['label'] = clicks_df.session_id.isin(buy_df.session_id)
print(clicks_df.head())

print(clicks_df.item_id.max() + 1)


class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseBinaryDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load('./data/processed.dataset')

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['../input/yoochoose_click_binary_1M_sess.dataset']

    def download(self):
        pass

    def process(self):

        data_list = []

        # process by session_id
        grouped = clicks_df.groupby('session_id')
        for session_id, group in tqdm(grouped):
            sess_item_id = LabelEncoder().fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id
            node_features = group.loc[group.session_id == session_id, ['sess_item_id', 'item_id']].sort_values('sess_item_id').item_id.drop_duplicates().values

            node_features = torch.LongTensor(node_features).unsqueeze(1)
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]

            edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
            x = node_features

            y = torch.FloatTensor([group.label.values[0]])

            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)

        data, slices = self.collate(data_list)
        torch.save((data, slices), './data/processed.dataset')



dataset = YooChooseBinaryDataset('./')
one_tenth_length = int(len(dataset) * 0.1)
dataset = dataset.shuffle()
train_dataset = dataset[:one_tenth_length * 8]
val_dataset = dataset[one_tenth_length * 8:one_tenth_length * 9]
test_dataset = dataset[one_tenth_length * 9:]
print(len(train_dataset), len(val_dataset), len(test_dataset))

embed_dim = 128
different_ids = 52707


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=different_ids, embedding_dim=embed_dim)
        #self.item_embedding = torch.nn.Embedding(num_embeddings=clicks_df.item_id.max() + 1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)

        x = F.relu(self.conv1(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)

        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x


def train(loader):
    model.train()

    loss_all = 0
    num_epochs = 30
    for epoch in range(num_epochs):
        print(epoch)
        for data in loader:
            data = data.to(device)
            optimizer.zero_grad()
            output = model(data)
            label = data.y.to(device)
            loss = crit(output, label)
            loss.backward()
            loss_all += data.num_graphs * loss.item()
            optimizer.step()
            #print(loss)
    return loss_all / len(train_dataset)


device = torch.device('cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()
train_loader = DataLoader(train_dataset, batch_size=512)
test_loader = DataLoader(test_dataset, batch_size=512)
val_loader = DataLoader(val_dataset, batch_size=512)




train(train_loader)


def evaluate(loader):
    model.eval()

    predictions = []
    labels = []
    print(len(loader))
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    #if len(loader) == 0:
    #     return
    predictions = np.hstack(predictions)
    labels = np.hstack(labels)

    return roc_auc_score(labels, predictions)


for epoch in range(1):
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)
    test_acc = evaluate(test_loader)
    print(epoch, train_acc, val_acc, test_acc)

Writing gnn.py


In [16]:
%%writefile model.py
import torch
from torch_geometric.nn import TopKPooling, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self, different_ids=52737, embed_dim=128):
        super(Net, self).__init__()

        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.5)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.5)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.5)
        self.item_embedding = torch.nn.Embedding(num_embeddings=different_ids, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)

        x = F.relu(self.conv1(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)

        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x


class Net2(torch.nn.Module):
    def __init__(self, different_ids=52737, embed_dim=512):
        super(Net2, self).__init__()

        self.item_embedding = torch.nn.Embedding(num_embeddings=different_ids, embedding_dim=embed_dim)

        self.conv1 = SAGEConv(embed_dim, 512)
        self.pool1 = TopKPooling(512, ratio=0.8)
        self.conv2 = SAGEConv(512, 512)
        self.pool2 = TopKPooling(512, ratio=0.8)
        self.conv3 = SAGEConv(512, 512)
        self.pool3 = TopKPooling(512, ratio=0.8)
        self.lin1 = torch.nn.Linear(1024, 512)
        self.lin2 = torch.nn.Linear(512, 256)
        self.lin3 = torch.nn.Linear(256, 128)
        self.lin4 = torch.nn.Linear(128, 64)
        self.lin5 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(256)
        self.bn2 = torch.nn.BatchNorm1d(128)
        self.bn3 = torch.nn.BatchNorm1d(64)
        self.act = torch.nn.ReLU()


    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)

        x = F.relu(self.conv1(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)

        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act(x)
        x = self.lin2(x)
        x = self.act(x)
        x = self.lin3(x)
        x = self.act(x)
        x = self.lin4(x)
        x = self.act(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin5(x)).squeeze(1)

        return x


Writing model.py


In [15]:
%%writefile preprocessing.py
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from sklearn.preprocessing import LabelEncoder
from torch_geometric.data import Data


datatset_size = 10000

# yoochoose-clicks.dat - Click events. Each record/line in the file has the following fields:
#     Session ID – the id of the session. In one session there are one or many clicks.
#     Timestamp – the time when the click occurred.
#     Item ID – the unique identifier of the item.
#     Category – the category of the item.
clicks_df = pd.read_csv('./data/yoochoose-clicks.dat', header=None)
clicks_df.columns = ['session_id', 'timestamp', 'item_id', 'category']

# filter out item session with length < 2
clicks_df['valid_session'] = clicks_df.session_id.map(clicks_df.groupby('session_id')['item_id'].size() > 2)
clicks_df = clicks_df.loc[clicks_df.valid_session].drop('valid_session',axis=1)
print(clicks_df.nunique())
print(clicks_df.head(5))

# yoochoose-buys.dat - Buy events. Each record/line in the file has the following fields:
#     Session ID - the id of the session. In one session there are one or many buying events.
#     Timestamp - the time when the buy occurred.
#     Item ID – the unique identifier of item.
#     Price – the price of the item.
#     Quantity – how many of this item were bought.
buy_df = pd.read_csv('./data/yoochoose-buys.dat', header=None)
buy_df.columns = ['session_id', 'timestamp', 'item_id', 'price', 'quantity']
print(buy_df.head())

# Encode item_id with values between 0 and n_classes - 1 (for embedding)
item_encoder = LabelEncoder()
clicks_df['item_id'] = item_encoder.fit_transform(clicks_df.item_id)
print(min(clicks_df['item_id']))
print(max(clicks_df['item_id']))
print(clicks_df.head())

# randomly sample a couple of them
sampled_session_id = np.random.choice(clicks_df.session_id.unique(), datatset_size, replace=False)
clicks_df = clicks_df.loc[clicks_df.session_id.isin(sampled_session_id)]
print(clicks_df.nunique())

# average length of session 
print(clicks_df.groupby('session_id')['item_id'].size().mean())

# add a boolean column named label to the clicks_df representing wether the click in the session is
# buy or not
clicks_df['label'] = clicks_df.session_id.isin(buy_df.session_id)

print(clicks_df.label.value_counts())
print(clicks_df.label.unique())
print(clicks_df.item_id.max() + 1)

data_list = []

# process by session_id
grouped = clicks_df.groupby('session_id')
for session_id, group in tqdm(grouped):
    sess_item_id = LabelEncoder().fit_transform(group.item_id)
    group = group.reset_index(drop=True)
    group['sess_item_id'] = sess_item_id
    node_features = group.loc[group.session_id == session_id, ['sess_item_id', 'item_id', 'timestamp']].sort_values('sess_item_id').item_id.drop_duplicates().values

    node_features = torch.LongTensor(node_features).unsqueeze(1)
    target_nodes = group.sess_item_id.values[1:]
    source_nodes = group.sess_item_id.values[:-1]

    edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
    x = node_features

    y = torch.FloatTensor([group.label.values[0]])

    data = Data(x=x, edge_index=edge_index, y=y)
    data_list.append(data)

torch.save(data_list, './data/processed.dataset')


Writing preprocessing.py


In [14]:
%%writefile test.py
import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score


def test(test_loader, model, device):
    model.eval()

    predictions = []
    labels = []
    print(len(test_loader))
    nb = len(test_loader)

    # Disable gradients
    pbar = tqdm(enumerate(test_loader), total=nb)
    for batch_idx, data in pbar:
        with torch.no_grad():
            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()

            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)

    #predictions = binarize(predictions, threshold=0.5, copy=True)
    #labels = binarize(labels, threshold=0.5, copy=True)



    return roc_auc_score(labels, predictions)

Overwriting test.py


In [13]:
%%writefile train.py
import os
import torch
from dataloader import LoadData
from torch_geometric.data import DataLoader
import argparse
from model import Net, Net2
from tqdm import tqdm
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from test import test
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter


wdir = 'weights' + os.sep  # weights dir
last = wdir + 'last.pt'
best = wdir + 'best.pt'
results_file = 'results.txt'


def train(args, model, device, train_loader, test_loader, val_loader, optimizer, save=True):

    # writer will output to ./runs/ directory by default
    tb_writer = SummaryWriter()

    # because this is a binary classification problem
    crit = torch.nn.BCELoss()

    # the number of batches
    nb = train_loader.__len__()

    # initialize the roc auc score
    best_roc_auc_score = 0
    start_epoch = 0

    if args.weights.endswith('.pt'):  # pytorch format

        chkpt = torch.load(args.weights, map_location=device)

        # load model
        try:
            model.load_state_dict(chkpt['model'], strict=False)
        except KeyError as e:
            print(e)
            exit()

        # load optimizer
        if chkpt['optimizer'] is not None:
            optimizer.load_state_dict(chkpt['optimizer'])
            best_roc_auc_score = chkpt['best_fitness']

        # load results
        if chkpt.get('training_results') is not None:
            with open(results_file, 'w') as file:
                file.write(chkpt['training_results'])  # write results.txt

        start_epoch = chkpt['epoch'] + 1
        del chkpt

    for epoch in range(start_epoch, args.epochs + 1):

        # ----------------------------------------------------------------
        # start epoch ----------------------------------------------------
        # ----------------------------------------------------------------

        print('\tEPOCH', epoch, '/', args.epochs)

        model.train()
        pbar = tqdm(enumerate(train_loader), total=nb)

        for batch_idx, data in pbar:

            # ----------------------------------------------------------------
            # start batch ----------------------------------------------------
            # ----------------------------------------------------------------

            data = data.to(device)
            # zero the gradient buffers
            optimizer.zero_grad()
            # classify batch
            output = model(data)
            label = data.y.to(device)
            loss = crit(output, label)
            print(loss)
            # loss = F.nll_loss(output, label)
            loss.backward()
            # update weights
            optimizer.step()

            if batch_idx % args.log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

            # ----------------------------------------------------------------
            # end batch ------------------------------------------------------
            # ----------------------------------------------------------------

        # Update scheduler
        # scheduler.step()

        final_epoch = epoch + 1 == args.epochs

        # Update best roc_auc_score
        roc_auc_score = test(test_loader, model, device)
        roc_auc_score2 = test(val_loader, model, device)
        print('test roc auc score:', roc_auc_score, "val", roc_auc_score2)

        # Write Tensorboard results
        if tb_writer:
            x = [roc_auc_score, roc_auc_score2]
            titles = ['ROC AUC TEST', 'ROC AUC VAL']
            for xi, title in zip(x, titles):
                tb_writer.add_scalar(title, xi, epoch)

        # Write epoch results
        with open(results_file, 'a') as f:
            f.write('%f' % roc_auc_score + '\n')

        if roc_auc_score > best_roc_auc_score:
            best_roc_auc_score = roc_auc_score

        # Save training results
        if save:
            with open(results_file, 'r') as f:
                # Create checkpoint
                chkpt = {'epoch': epoch,
                         'best_fitness': best_roc_auc_score,
                         'training_results': f.read(),
                         'model': model.state_dict(),
                         'optimizer': None if final_epoch else optimizer.state_dict()}

            # Save last checkpoint
            torch.save(chkpt, last)

            # Save best checkpoint
            if round(best_roc_auc_score, 2) == round(roc_auc_score, 2):
                torch.save(chkpt, best)

            # Delete checkpoint
            del chkpt

    # ----------------------------------------------------------------
    # end epoch ------------------------------------------------------
    # ----------------------------------------------------------------


def main():
    parser = argparse.ArgumentParser(description='PyTorch Battery')

    parser.add_argument('--resume', action='store_true',
                        help='resume training from last.pt')
    parser.add_argument('--batch-size', type=int, default=512, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--epochs', type=int, default=400, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--weights', type=str, default='',
                        help='initial weights path')

    args = parser.parse_args()

    args.weights = last if args.resume else args.weights

    print('\n\n\tTrain...\n')

    use_cuda = not args.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    dataset = LoadData('./data/processed.dataset')

    one_tenth_length = int(len(dataset) * 0.001)
    train_dataset = dataset[:one_tenth_length * 8]
    val_dataset = dataset[one_tenth_length * 8:one_tenth_length * 9]
    test_dataset = dataset[one_tenth_length * 9:]

    print(len(train_dataset), len(val_dataset), len(test_dataset))

    train_loader = DataLoader(train_dataset,
                              batch_size=args.batch_size,
                              num_workers=1,
                              shuffle=True,
                              pin_memory=True)

    val_loader = DataLoader(val_dataset,
                            batch_size=args.batch_size,
                            num_workers=1,
                            shuffle=True,
                            pin_memory=True)

    test_loader = DataLoader(test_dataset,
                             batch_size=args.batch_size,
                             num_workers=1,
                             shuffle=True,
                             pin_memory=True)

    train(args, model, device, train_loader, test_loader, val_loader, optimizer)


if __name__ == "__main__":
    main()

Writing train.py


In [ ]:
!python3 preprocessing.py

sys:1: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.


In [19]:
!python3 train.py




	Train...

Traceback (most recent call last):
  File "train.py", line 209, in <module>
    main()
  File "train.py", line 178, in main
    dataset = LoadData('./data/processed.dataset')
  File "/content/dataloader.py", line 9, in __init__
    self.data = torch.load(data_path)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 594, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: './data/processed.dataset'
